# **1) Initiall Instructions**

In [ ]:
!pip install -q rdkit

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, matthews_corrcoef

from rdkit import Chem
from rdkit.Chem.AllChem import GetMorganGenerator
from rdkit.Chem.SaltRemover import SaltRemover

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

tf.keras.utils.set_random_seed(42)

# **2) Data Preparation**

In [ ]:
#Path to DataFrame containing parsed molecules from ChEMBL
data = pd.read_csv(r'/content/drive/MyDrive/CNNDock_2__poprawki/path', usecols=['Molecule ChEMBL ID', 'Smiles', 'Bin_Activity'])
df = pd.DataFrame(data)

list_of_esters = ['CHEMBL4846931', 'CHEMBL4849680', 'CHEMBL4863967', 'CHEMBL4867606', 'CHEMBL4874372', 'CHEMBL5431767', 'CHEMBL5430776', 'CHEMBL5412404', 'CHEMBL5398411', 'CHEMBL5438068']
df = df[~df['Molecule ChEMBL ID'].isin(list_of_esters)]

In [ ]:
#Fingerprint generation and salt removal
morgan_gen = GetMorganGenerator(fpSize=8192, radius=5)
remover = SaltRemover()

fps_list = []
for smiles in df['Smiles']:
  mol = Chem.MolFromSmiles(smiles)
  mol = remover.StripMol(mol)
  fps_list.append(morgan_gen.GetFingerprint(mol))

df['fps'] = fps_list

In [ ]:
"""In order to keep the datasets equal, it is necessary to perform the intersection of image sets"""

#Path to images after docking
path_dock = r'/content/drive/MyDrive/CNNDock_2__poprawki/datasety_od_nowa/dokowanie/images_docking_sprawdzona_rozdzielczosc/images_docking_sprawdzona_rozdzielczosc'

#Path to images after DFT optimization
path_quanta = r'/content/drive/MyDrive/CNNDock_2__poprawki/datasety_od_nowa/kwanty/images_quanta/images_dft'

list_dock = [elem.split('_')[0] for elem in os.listdir(path_dock)]
list_quanta = [elem.split('_')[0] for elem in os.listdir(path_quanta)]

print(len(list_dock))
print(len(list_quanta))

common_idx = list(set(list_dock) & set(list_quanta))
print(len(common_idx))

In [ ]:
#Labels, fingerprints and ChEMBL IDs extraction, which is consistent with above intersection
labels_list = df[df['Molecule ChEMBL ID'].isin(common_idx)]['Bin_Activity'].tolist()
fps_list = df[df['Molecule ChEMBL ID'].isin(common_idx)]['fps'].tolist()
chembl_list = df[df['Molecule ChEMBL ID'].isin(common_idx)]['Molecule ChEMBL ID'].tolist()

labels_arr = np.array(labels_list)
fps_arr = np.array(fps_list)

print(labels_arr.shape)
print(fps_arr.shape)

# **3) 5-Fold Crossvalidation**

In [ ]:
#5-fold crossvalidation
callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
fold_no = 1
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

all_folds_scores_history = {'val_acc': [], 'mcc': []}

for train, test in skfold.split(fps_arr, labels_arr):
  x_fps_input = keras.layers.Input(shape=(8192,))

  x_fps = keras.layers.Dense(128, activation='relu')(x_fps_input)
  x_fps = keras.layers.Dropout(0.3)(x_fps)
  x_fps = keras.layers.Dense(64, activation='relu')(x_fps)
  x_fps = keras.layers.Dropout(0.3)(x_fps)
  x_fps = keras.layers.Dense(32, activation='relu')(x_fps)
  x_fps = keras.layers.Dropout(0.3)(x_fps)
  output = keras.layers.Dense(1, activation='sigmoid')(x_fps)

  model = keras.models.Model(inputs=x_fps_input, outputs=output,)
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



  print(f'Fold {fold_no} ...')

  history = model.fit(
    fps_arr[train], labels_arr[train],
    batch_size=32,
    epochs=20,
    verbose=0,
    callbacks=[callback],
    validation_data=(fps_arr[test], labels_arr[test])
    )

  mean_acc_per_fold = np.mean(history.history['val_accuracy'])
  all_folds_scores_history['val_acc'].append(mean_acc_per_fold)

  y_pred_probs = model.predict(fps_arr[test])
  y_pred = (y_pred_probs > 0.5).astype(int)
  mcc = matthews_corrcoef(labels_arr[test], y_pred)
  all_folds_scores_history['mcc'].append(mcc)

  fold_no += 1

print()
print(f'Average Accuracy for 5-fold crossvalidation: {np.mean(all_folds_scores_history["val_acc"]):.2f}')
print(f'Average MCC for 5-fold crossvalidation: {np.mean(all_folds_scores_history["mcc"]):.2f}')


# **4) Model Builiding, Training and Prediction**

In [ ]:
#Data splitting (train set and test set) and training (with EarlyStopping to prevent overfitting)
X_train, X_test, y_train, y_test, chembl_train, chembl_test = train_test_split(fps_arr, labels_arr, chembl_list, test_size=0.2, random_state=42, stratify=labels_list)
callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

In [ ]:
#Creating a model and training
x_fps_input = keras.layers.Input(shape=(8192,))

x_fps = keras.layers.Dense(128, activation='relu')(x_fps_input)
x_fps = keras.layers.Dropout(0.3)(x_fps)
x_fps = keras.layers.Dense(64, activation='relu')(x_fps)
x_fps = keras.layers.Dropout(0.3)(x_fps)
x_fps = keras.layers.Dense(32, activation='relu')(x_fps)
x_fps = keras.layers.Dropout(0.3)(x_fps)
output = keras.layers.Dense(1, activation='sigmoid')(x_fps)

model = keras.models.Model(inputs=x_fps_input, outputs=output,)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), callbacks=[callback])

In [ ]:
#Training visualizations
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
ax[0].plot(history.history['accuracy'], label='Training Accuracy')
ax[0].plot(history.history['val_accuracy'], label='Validation Accuracy')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy')
ax[0].set_title('Training and Validation Accuracy')
ax[0].legend()


ax[1].plot(history.history['loss'], label='Training Loss')
ax[1].plot(history.history['val_loss'], label='Validation Loss')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Loss')
ax[1].set_title('Training and Validation Loss')
ax[1].legend()

plt.legend()
plt.tight_layout()
plt.show()



In [ ]:
#Test set labels prediction
y_pred = model.predict(X_test, batch_size=32)
y_pred = np.where(y_pred > 0.5, 1, 0)

test_acc = accuracy_score(y_test, y_pred)
test_classification_report = classification_report(y_test, y_pred)
test_confusion_matrix = confusion_matrix(y_test, y_pred)
test_mcc = matthews_corrcoef(y_test, y_pred)

print(f'Test Accuracy: {test_acc}')
print(f'Test MCC: {test_mcc}')
print(f'Test Classification Report:\n{test_classification_report}')

In [ ]:
#Test set labels prediction - confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(test_confusion_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.text(0, 2.4, f'Test accuracy: {100*test_acc:.2f}%', fontsize=15)
plt.text(0, 2.5, f'Test mcc: {test_mcc:.2f}', fontsize=15)

In [ ]:
#Labels comparison (true vs predicted for test set)
df = pd.DataFrame({
    'chembl_id': chembl_test,
    'true': y_test,
    'pred': y_pred.flatten()
})

for chembl_id, group in df.groupby('chembl_id'):
    true_vals = group['true'].values
    pred_vals = group['pred'].values
    if not np.array_equal(true_vals, pred_vals):
        print(f"Chembl ID: {chembl_id}.Bad predictions {30*'x'}")
    print(f"{chembl_id}")
    print(f"True:      {true_vals}")
    print(f"Predicted: {pred_vals}")
    print("-" * 30)

In [ ]:
"""This commented cell allows user to save trained model to particular directory"""

# os.chdir('/content/drive/MyDrive/pathtosave')

# cnnmodel = model
# path = os.path.join(os.getcwd(), 'model.joblib')
# dump(cnnmodel, path)